In [56]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.articles

In [12]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Kelly\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [22]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [26]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')

# loop over results to get article data
for result in results:
     
#     try:
    # scrape title
    news_title = result.find('div', class_='content_title').text
    
    # scrape paragraph text
    news_p = result.find('div', class_='article_teaser_body').text
    
    # print article data
    print(news_title)
    print(news_p)
    
    # Dictionary to be inserted into MongoDB
    post = {
        'title': news_title,
        'paragraph': news_p,
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

#   browser.links.find_by_partial_text('MORE').click()
# except ElementDoesNotExist:
#     print("Scraping Complete")

Nearly 11 Million Names of Earthlings are on Mars Perseverance
When the Perseverance rover safely touched down on the Martian surface, inside Jezero Crater, on Feb. 18, 2021, it was also a safe landing for the nearly 11 million names on board.
NASA's Mars Perseverance Rover Provides Front-Row Seat to Landing, First Audio Recording of Red Planet 
The agency’s newest rover captured first-of-its kind footage of its Feb. 18 touchdown and has recorded audio of Martian wind.


NASA to Reveal New Video, Images From Mars Perseverance Rover
First-of-its kind footage from the agency’s newest rover will be presented during a briefing this morning.
NASA's Mars Helicopter Reports In 
The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. 
NASA's Perseverance Rover Sends Sneak Peek of Mars Landing
The six-wheeled robot’s latest data since touching down yesterday include a hi-res image captured as the rover’s jetpack lowered it to the ground.

In [ ]:
# notes for later
# for x in range
# look for first a tag
# find href
# browser.back()

In [35]:
# def featured_image(browser):
#     # Visit URL
#     url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
#     browser.visit(url)

#     # Find and click the full image button
#     full_image_elem = browser.find_by_tag('button')[1]
#     full_image_elem.click()

#     # Parse the resulting html with soup
#     html = browser.html
#     img_soup = soup(html, 'html.parser')

#     # Add try/except for error handling
#     try:
#         # find the relative image url
#         img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

#     except AttributeError:
#         return None

#     # Use the base url to create an absolute url
#     img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'

#     return img_url

In [47]:
# URL of page to be scraped
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(url)

# Find and click the full image button
browser.click_link_by_partial_text('FULL IMAGE')

browser.is_element_present_by_text('more info', wait_time=1)

html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
img_soup = BeautifulSoup(html, 'html.parser')

In [48]:
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
print(img_url_rel)

image/featured/mars3.jpg


In [49]:
img_url = url + img_url_rel
img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.htmlimage/featured/mars3.jpg'

In [58]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'

table = pd.read_html(url)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [60]:
html_table = table.to_html

AttributeError: 'list' object has no attribute 'to_html'